pip install docx2txt


pip install wikipedia -q

In [1]:
from IPython.display import display, Markdown

In [2]:
import getpass
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI


In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv('./.env')

True

def load_document(file): \
    from langchain.document_loaders import PyPDFLoader\
    print(f'Loading {file}')\
    loader = PyPDFLoader(file)\
    data = loader.load()\
    return data

In [ ]:
#print(data[1].page_content)\
#print(data[0].metadata) #page numbering starts with 0\
#print(f'you have {len(data)} pages in your data')\
#print(f'There are {len(data[20].page_content)} characters in your page')

In [4]:
def load_document(file):
    name, extension = os.path.splitext(file)
    
    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
        
    else:
        print("Unsupported Format")
        return(None)
    
    data = loader.load()
    return data

#### Running the Code

In [5]:
data1 = load_document("D:/Books/Final Year 1st_Sem/Industrial Training/HPPWD.docx")

Loading D:/Books/Final Year 1st_Sem/Industrial Training/HPPWD.docx


print(data1[0].page_content)

### Public and Private Service Loaders

In [6]:
data = load_document("D:/LangChain courses/LLM Project/[CourseClub.Me] ZerotoMastery - Build an LLM-powered QA App using LangChain, OpenAI and Python/code/Notebooks/files/us_constitution.pdf")

Loading D:/LangChain courses/LLM Project/[CourseClub.Me] ZerotoMastery - Build an LLM-powered QA App using LangChain, OpenAI and Python/code/Notebooks/files/us_constitution.pdf


In [7]:
type(data)

list

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 256,
    chunk_overlap=0,
    length_function=len
)


#### .create_documents()
 is used when starting with raw text and creating Document objects from it.
#### .split_documents()
is used when you already have Document objects and want to split their content into smaller chunks.

chunks = text_splitter.create_documents([str(data)])\
print(chunks)

In [9]:
chunks = text_splitter.split_documents(data)


In [10]:
print(len(chunks))

190


In [11]:
print(chunks[10].page_content)

Representatives
shall
chuse
their
Speaker
and
other
Of ficers;and
shall
have
the
sole
Power
of
Impeachment.
Section
3:
The
Senate
The
Senate
of
the
United
States
shall
be
composed
of
two
Senators
from
each
State,
chosen
by
the
Legislature
thereof,
for
six


In [12]:
import tiktoken

In [13]:
def print_embedding_cost(texts):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 *0.0004:.6f}')

print_embedding_cost(chunks)

Total Tokens: 16711
Embedding Cost in USD: 0.006684


#### Embedding and Uploading

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [15]:
import pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.environ.get('PINECONE_KEY'))

In [16]:
def insert_embeddings(index_name):
    if index_name in pc.list_indexes():
        print(f'Index {index_name} already exists. loading embeddings.', end = '')
        vector_store = pc.from_existing_index(index_name, embeddings)
        print('done')
    else:
        print(f'Creating index {index_name} and embeddings.',end = '')
        vector_store =  pc.create_index(index_name, dimension=768,metric='cosine',
        spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"))
        index = pc.Index(index_name)
        embeddings = [gemini_embeddings.embed_query(doc.page_content) for doc in chunks]
        vectors = []
        for i, embedding in enumerate(embeddings):
            chunk_id = f"chunk_{i}"
            metadata = {"text": chunks[i].page_content}
            vectors.append((chunk_id, embedding, metadata))
            index.upsert(vectors)
        

In [17]:
def delete_pinecone_index(index_name = 'all'):
    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes..')
        for index in indexes:
            pc.delete_index(index)
            print('done')
    else:
        print(f' Deleting index {index_name} ... ', end = '')
        pc.delete_index(index_name)
        print('done')

In [18]:
delete_pinecone_index()


Deleting all indexes..


In [19]:
index_name = 'askadocument'
vector_store = insert_embeddings(index_name)

Creating index askadocument and embeddings.

#### Asking and Getting Answers

In [20]:
gemini_api_key = os.environ.get('GOOGLE_API_KEY')

In [21]:
genai.configure(api_key=gemini_api_key)

In [22]:
index = pc.Index(index_name)

In [23]:
from langchain.vectorstores import Pinecone


vector_store = Pinecone(
    index=index, 
    embedding=gemini_embeddings,  
    text_key="text"  
)

D:\Anaconda\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(


In [24]:
type(vector_store)

langchain_community.vectorstores.pinecone.Pinecone

In [25]:
from langchain.chains import RetrievalQA
def aq(query):
    values = {"model": "gemini-1.5-flash-latest"}  # or another supported model
    llm = ChatGoogleGenerativeAI(**values)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs ={'k' : 3})
    chain = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever=retriever)
    ans = chain.run(query)
    return ans

In [26]:
q = "What is the context of document ?"


In [27]:
display(Markdown(aq(q)))

D:\Anaconda\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


The text you provided is a snippet from the **United States Constitution**, specifically from **Article III, Section 2**, which deals with the **judicial power** of the United States. 

The part you included focuses on the jurisdiction of the **Supreme Court**. It outlines the types of cases the court has original jurisdiction over, including:

* **Cases affecting ambassadors, other public ministers and consuls**
* **Cases of admiralty and maritime jurisdiction**
* **Cases in which a state is a party** 
* **Controversies between two or more states**
* **Controversies between citizens of different states**
* **Controversies between citizens of the same state claiming lands under grants of different states**
* **Controversies between a state, or the citizens thereof, and foreign states, citizens, or subjects**

The rest of the document discusses the appellate jurisdiction of the Supreme Court and the power of Congress to create inferior courts. 


In [28]:
import time
i = 1
print('Write quit or exit to leave terminal')
while True:
    q = input(f'Question #{i}: ')
    i= i + 1
    if q.lower() in ['quit', 'exit']:
        print('Leaving terminal...')
        time.sleep(2)
        break
    
    answer = aq(q)  # Get the answer from the function
    display(Markdown(answer))  # Display the answer as Markdown  # Print the answer as plain text
    print(f'\n{"-"*50}\n')

Write quit or exit to leave terminal
Question #1: what is said about citizens


The provided text states that citizens of the United States are subject to the jurisdiction of the United States and the State wherein they reside. It also says that no State shall make or enforce any law that would abridge the privileges or immunities of citizens of the United States. 

Additionally, it mentions that citizens of each State are entitled to all Privileges and Immunities of Citizens in the several States. 



--------------------------------------------------

Question #2: quit
Leaving terminal...


In [29]:
def load_from_wikipedia(query,lang = 'en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=2)
    data = loader.load()
    return data

In [30]:
data = load_from_wikipedia("Kolkata Case")
print(data[0].page_content)

On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College and Hospital in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate about the safety of women and doctors in India, and has sparked significant outrage, nationwide and international protests, and demands for a thorough investigation.


== Incident ==
On the night of 8 August 2024, the victim had dinner with her colleagues and retired in a seminar hall after a 36-hour shift in the hospital. At about 9:30 IST the following morning, her body was discovered in the seminar hall in a semi-nude state with her eyes, mouth, and genitals bleeding. She was later declared dead. College authorities initially informed her family that she had committed suicide. 
An autopsy was conducted on 9 August and, after its completion, an unnatural death case was registered.


== Investigation ==


=== Autopsy report ===
An auto

In [31]:
data[0]

Document(metadata={'title': '2024 Kolkata rape and murder incident', 'summary': 'On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College and Hospital in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate about the safety of women and doctors in India, and has sparked significant outrage, nationwide and international protests, and demands for a thorough investigation.', 'source': 'https://en.wikipedia.org/wiki/2024_Kolkata_rape_and_murder_incident'}, page_content='On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College and Hospital in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate about the safety of women and doctors in India, and has sparked significant outrage, nationwide and international protests, and demands for a thoro

In [32]:
data

[Document(metadata={'title': '2024 Kolkata rape and murder incident', 'summary': 'On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College and Hospital in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate about the safety of women and doctors in India, and has sparked significant outrage, nationwide and international protests, and demands for a thorough investigation.', 'source': 'https://en.wikipedia.org/wiki/2024_Kolkata_rape_and_murder_incident'}, page_content='On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College and Hospital in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate about the safety of women and doctors in India, and has sparked significant outrage, nationwide and international protests, and demands for a thor

In [35]:
type(data)

list

In [33]:
delete_pinecone_index()

Deleting all indexes..
done


In [34]:
chunks = text_splitter.split_documents(data)

In [35]:
chunks[0].page_content

'On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College and Hospital in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate'

In [36]:
index_name = "gemini"
vector_store = insert_embeddings(index_name)

Creating index gemini and embeddings.

In [37]:
index = pc.Index(index_name)

In [38]:
from langchain.vectorstores import Pinecone

# Initialize the Pinecone vector store
vector_store = Pinecone(
    index=index,  # Your Pinecone index object
    embedding=gemini_embeddings,  # Embedding function
    text_key="text"  # Metadata key where document text is stored
)

In [39]:
type(vector_store)

langchain_community.vectorstores.pinecone.Pinecone

In [40]:
from langchain.chains import RetrievalQA
def aq(query):
    values = {"model": "gemini-1.5-flash-latest"}  # or another supported model
    llm = ChatGoogleGenerativeAI(**values)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs ={'k' : 3})
    chain = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever=retriever)
    ans = chain.run(query)
    return ans

In [41]:
q = 'what happened at seminar room'

In [42]:
display(Markdown(aq(q)))

The victim's body was discovered in the seminar room in a semi-nude state with her eyes, mouth, and genitals bleeding. She was later declared dead. 


In [43]:
q = 'what is written college authorities'

In [44]:
display(Markdown(aq(q)))

The provided text states that "College authorities initially informed her family that she had committed suicide." 


### Creating Memory

In [45]:
def ask_with_memory(vector_store, ques, chat_history = []):
    from langchain.chains import ConversationalRetrievalChain
    values = {"model": "gemini-1.5-flash-latest"}
    llm = ChatGoogleGenerativeAI(**values)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs ={'k' : 3})
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question':ques,'chat_history':chat_history})
    chat_history.append((ques, result['answer']))
        
    return result, chat_history

In [46]:
chat_history = []
ques = 'What was the date'
result, chat_history = ask_with_memory(vector_store, ques, chat_history)
print(result['answer'])

D:\Anaconda\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


The provided text mentions that the individual was sent to 14-day judicial custody on August 23rd. 



In [47]:
ques = 'multiply it by 2'
result, chat_history = ask_with_memory(vector_store, ques, chat_history)
print(result['answer'])

August 23rd is a date, not a number.  I can't multiply a date by 2. 

